# Student: Rafael Santos  #HomeWork 4 - PyCity

#Data Analystics and Visualization Cohort 3

# PyCity Schools - Overall Analysis
---
* 1) Schools that spend up to USD 615 per student had overall passing rates 14% to 21% higher than those schools spending more than USD 615 up to USD 675 per student.

* 2) Small and medium schools have significantly higher passing rates (95% and 88% respectively) than larger schools (74%), suggesting that the number of students per school may negatively influence the quality of the student's performance.

* 3) Charter schools have overall passing rate 11% higher than District schools, with the 67% passing math rate clearly determining District schools overall passing rate of 73%, just above average.

* 4) All the top 10 performing schools are charter and have budget per studend smaller than the bottom 10 performing schools, which are district type. This confirms that budget is not a leading factor associated to higher performance of schools, and suggests that one of the opportunities with district schools is to review the way their budget is utilized, potentially by leveraging best practices from top performing charter schools.

* 5) Based on this dataset analysis, gender and grade did not seem to influence performance, showing students had relatively similar passing scores and rates. 

---

### BEGIN OF THE MAIN CODE ###

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])

# Checking a sample of the data to understand its structure
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


### @@User defined functions

In [2]:
#user-defined function to merge tables
def merging(x,y,z):   
    check = y.name
    y = y.reset_index(level=['school_name'])        #convert index into column to allow merging
    x = pd.merge(x,y,how="left",on=['school_name']) #merge x and y on columns with name 'school_name'
    x = x.rename(columns={check:z})                 #rename original merged column name with  z
    x = x.rename(columns={0:z})                     #rename merged column with "0" name as z

    return x


#user-defined function to format frequent numeric data fields for cleaner presentation
def formarter(x,y):
    if y == 0:
        x['Average Math Score'] = x['Average Math Score'].map("{:.2f}".format)
        x['Average Reading Score'] = x['Average Reading Score'].map("{:.2f}".format)
        x['% Passing Math'] = x['% Passing Math'].map("{:.2%}".format)
        x['% Passing Reading'] = x['% Passing Reading'].map("{:.2%}".format)
        x['% Overall Passing Rate'] = x['% Overall Passing Rate'].map("{:.2%}".format)
    else:
        x['Budget'] = x['Budget'].map("${:,.2f}".format)
        x['Total Students'] = x['Total Students'].map("{:,.0f}".format)

    return x

## 1) District Summary

* Calculate the total number of schools

In [3]:
TotalSchools = school_data_complete["school_name"].nunique()
TotalSchools

15

* Calculate the total number of students

In [4]:
TotalStudents = school_data_complete["student_name"].count()
TotalStudents

39170

* Calculate the total budget

In [5]:
TotalBudget = school_data_complete.loc[school_data_complete['budget'] > 0,['school_name','budget']]

#Remove duplicate budget data for each school
TotalBudget.drop_duplicates(subset="school_name", keep = 'first', inplace = True)

#Sum total budget for the district
TotalBudget = TotalBudget["budget"].sum()  #24649428
TotalBudget

24649428

* Calculate the average math score 

In [6]:
AvgMathScore = school_data_complete["math_score"].mean() #78.98537145774827
AvgMathScore

78.98537145774827

* Calculate the average reading score

In [7]:
AvgReadingScore = school_data_complete["reading_score"].mean() #81.87784018381414
AvgReadingScore

81.87784018381414

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

In [8]:
OvrPassScore = (AvgMathScore + AvgReadingScore)/2
OvrPassScore

80.43160582078121

* Calculate the percentage of students with a passing math score (70 or greater)

In [9]:
#Calculate the percentage of students with a passing math score (70 or greater)
AvgPercentageStudentMath70higher = school_data_complete.loc[school_data_complete['math_score'] >= 70,\
                                                        ['student_name', 'math_score']]

AvgPercentageStudentMath70higher = (len(AvgPercentageStudentMath70higher)/TotalStudents)
AvgPercentageStudentMath70higher

0.749808526933878

* Calculate the percentage of students with a passing reading score (70 or greater)

In [10]:
AvgPercentageStudentRead70higher = school_data_complete.loc[school_data_complete['reading_score'] >= 70,\
                                                        ['student_name','reading_score']]

AvgPercentageStudentRead70higher = (len(AvgPercentageStudentRead70higher)/TotalStudents)
AvgPercentageStudentRead70higher

0.8580546336482001

* (Added) calculate the % average passing rate for the district - consistent with other tables/analysis

In [11]:
OvrPassRate = (AvgPercentageStudentMath70higher + AvgPercentageStudentRead70higher)/2
OvrPassRate

0.8039315802910391

* Create a dataframe to hold the above results

In [12]:
summary = pd.DataFrame([  {"% Overall Passing Rate":OvrPassRate,
                           "% Passing Math":AvgPercentageStudentMath70higher,
                           "% Passing Reading":AvgPercentageStudentRead70higher,
                           "Overall Passing Score":OvrPassScore,
                           "Average Math Score":AvgMathScore,
                           "Average Reading Score":AvgReadingScore,
                           "Total Schools":TotalSchools,
                           "Total Students":TotalStudents,
                           "Budget":TotalBudget
                          } ])

* Optional: give the displayed data cleaner formatting

In [13]:
#Format numeric values
formarter(summary,0)
formarter(summary,1)
summary['Total Schools'] = summary['Total Schools'].map("{:,.0f}".format)
summary['Overall Passing Score'] = summary['Overall Passing Score'].map("{:,.2f}".format)

#Rename index
summary = summary.set_index([pd.Index(["District-Level Summary:"])])

#Reorder columns
summary = summary[["Total Students","Total Schools","Budget","Average Reading Score","Average Math Score",
                   "Overall Passing Score","% Passing Reading","% Passing Math","% Overall Passing Rate",]]

### @@District Summary - Results:

In [14]:
summary

,Total Students,Total Schools,Budget,Average Reading Score,Average Math Score,Overall Passing Score,% Passing Reading,% Passing Math,% Overall Passing Rate
District-Level Summary:,"39,170",15,"$24,649,428.00",81.88,78.99,80.43,85.81%,74.98%,80.39%


## 2) School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Average Math Score
  * Average Reading Score
  * Total Students

In [15]:
#found out researching that I could obtain many of the values at once with using pandas.pivot_table

table = pd.pivot_table(school_data_complete,values =['student_name','math_score','reading_score'],\
                       index=['school_name','type'],\
                       aggfunc={"student_name": len, 'math_score':np.mean,'reading_score': np.mean})

#checking how the table looks like. It comes with two index columns.
display (table.head(), table.describe())

,,math_score,reading_score,student_name
school_name,type,,,
Bailey High School,District,77.048432,81.033963,4976
Cabrera High School,Charter,83.061895,83.975780,1858
Figueroa High School,District,76.711767,81.158020,2949
Ford High School,District,77.102592,80.746258,2739
Griffin High School,Charter,83.351499,83.816757,1468


,math_score,reading_score,student_name
count,15.000000,15.000000,15.000000
mean,80.432530,82.529188,2611.333333
std,3.374548,1.519329,1420.915282
min,76.629414,80.744686,427.000000
25%,77.060448,81.000178,1698.000000
50%,83.061895,83.725724,2283.000000
75%,83.388902,83.901965,3474.000000
max,83.839917,84.044699,4976.000000


@@@@@@@@@@@@@
Side analysis/sample of exploration done - a.k.a "not needed for the code to be executed"


In [16]:
#Double Checking/Isolating data with another method, confirming consistency with PivotTable numbers. 

##Total Students 
SchoolStudents = school_data_complete.groupby('school_name')['student_name'].count() 

##Average Math Score
AverageMathScore = school_data_complete.groupby('school_name')['math_score'].mean() 

#Average Reading Score        
AverageReadingScore = school_data_complete.groupby('school_name')['reading_score'].mean()

#it is possible to confirm both methods (Pivot Table and GroupBy) worked.
display(SchoolStudents, AverageMathScore, AverageReadingScore)

school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: student_name, dtype: int64

school_name
Bailey High School       77.048432
Cabrera High School      83.061895
Figueroa High School     76.711767
Ford High School         77.102592
Griffin High School      83.351499
Hernandez High School    77.289752
Holden High School       83.803279
Huang High School        76.629414
Johnson High School      77.072464
Pena High School         83.839917
Rodriguez High School    76.842711
Shelton High School      83.359455
Thomas High School       83.418349
Wilson High School       83.274201
Wright High School       83.682222
Name: math_score, dtype: float64

school_name
Bailey High School       81.033963
Cabrera High School      83.975780
Figueroa High School     81.158020
Ford High School         80.746258
Griffin High School      83.816757
Hernandez High School    80.934412
Holden High School       83.814988
Huang High School        81.182722
Johnson High School      80.966394
Pena High School         84.044699
Rodriguez High School    80.744686
Shelton High School      83.725724
Thomas High School       83.848930
Wilson High School       83.989488
Wright High School       83.955000
Name: reading_score, dtype: float64

@@@@@@@@@@@@@ Decided to keep the pivot table (reducing the number of merges) and calculate other fields using the same approach.

* Total School Budget

In [17]:
SchoolBudget=school_data_complete.groupby('school_name')['budget'].mean()
SchoolBudget.describe()

count    1.500000e+01
mean     1.643295e+06
std      9.347763e+05
min      2.480870e+05
25%      1.046265e+06
50%      1.319574e+06
75%      2.228999e+06
max      3.124928e+06
Name: budget, dtype: float64

* Per Student Budget

In [18]:
PerStudentSchoolBudget = SchoolBudget/SchoolStudents
PerStudentSchoolBudget.describe()

count     15.000000
mean     620.066667
std       28.544368
min      578.000000
25%      591.500000
50%      628.000000
75%      641.500000
max      655.000000
dtype: float64

  * % Passing Math

In [19]:
AverageMathScore = school_data_complete.loc[school_data_complete['math_score']>=70,['school_name','math_score']]
PassingMathStudendsPerSchool = AverageMathScore.groupby('school_name')['math_score'].count()/SchoolStudents
PassingMathStudendsPerSchool.describe()

count    15.000000
mean      0.809871
std       0.139988
min       0.656839
25%       0.665233
50%       0.925059
75%       0.936297
max       0.945946
dtype: float64

  * % Passing Reading

In [20]:
AverageReadingScore = school_data_complete.loc[school_data_complete['reading_score']>=70,['school_name','reading_score']]
PassingReadingStudendsPerSchool = AverageReadingScore.groupby('school_name')['reading_score'].count()/SchoolStudents 
PassingReadingStudendsPerSchool.describe()

count    15.000000
mean      0.892190
std       0.081807
min       0.792990
25%       0.810427
50%       0.958546
75%       0.965754
max       0.973089
dtype: float64

* Overall Passing Rate (Average of the above two)

In [21]:
OverallPassingRatePerSchool=(PassingMathStudendsPerSchool + PassingReadingStudendsPerSchool)/2
OverallPassingRatePerSchool.describe()

count    15.000000
mean      0.851030
std       0.110716
min       0.732933
25%       0.737221
50%       0.943794
75%       0.952347
max       0.955867
dtype: float64

* Create an overview table that summarizes key metrics about each school:
---
* @@@ by looking at the description of the data and headers, and after running into several errors, I noticed:
* * * 1) datasets had the same lenght 
* * * 2) some had names in the columns as I sliced the data, others did have name.
* * * 3) indexes for the pivot table and other tables were different
---
* @@@ Identified opportunity for:
* * * 4) making column header names consistent across all tables
* * * 5) making index consistent across all tables
* * * 6) leverage the use of functions for both merging and formating repetitive lines of code
* * * 7) creating a full deep copy of the main table to be used (without formatting) later on other calculations.

In [22]:
#merging all the data calculated in multiple columns into one table:

table = table.reset_index(level=['type'])         #convert pivot table indexes into columns                   
table = table.reset_index(level=['school_name'])  #convert pivot table indexes into columns

table = table.rename(columns={'type':'Type'})
table = table.rename(columns={'student_name':'Total Students'})
table = table.rename(columns={'math_score':'Average Math Score'})
table = table.rename(columns={'reading_score':'Average Reading Score'})

#merging tables using user-defined function "merging"
schoolsummarytable = merging(table, SchoolBudget,'Budget')
schoolsummarytable = merging(schoolsummarytable, PerStudentSchoolBudget,'Per Student Budget')
schoolsummarytable = merging(schoolsummarytable, PassingMathStudendsPerSchool,'% Passing Math')
schoolsummarytable = merging(schoolsummarytable, PassingReadingStudendsPerSchool,'% Passing Reading')
schoolsummarytable = merging(schoolsummarytable, OverallPassingRatePerSchool,'% Overall Passing Rate')


#reordering the collumn
schoolsummarytable = schoolsummarytable[["school_name", "Type", "Total Students",
                                         "Budget","Per Student Budget",
                                         "Average Reading Score","Average Math Score",
                                         "% Passing Reading","% Passing Math",
                                         "% Overall Passing Rate",]]

#make a Full Deep Copy for use in other tables, without formatting the numeric results ($, %, x,xxx.00, etc)
df_temp = schoolsummarytable.copy(deep=True)

#format the results for the numeric values using user-defined function "formarter"
formarter(schoolsummarytable,0)
formarter(schoolsummarytable,1)

#format table-specific/unique fields
schoolsummarytable['Per Student Budget'] = schoolsummarytable['Per Student Budget'].map("${:,.2f}".format)
schoolsummarytable = schoolsummarytable.rename(columns={'school_name':'School Name'})

### @@School Summary - Results:

* Create an overview table that summarizes key metrics about each school:

In [23]:
schoolsummarytable

,School Name,Type,Total Students,Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,81.03,77.05,81.93%,66.68%,74.31%
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.98,83.06,97.04%,94.13%,95.59%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.16,76.71,80.74%,65.99%,73.36%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,80.75,77.10,79.30%,68.31%,73.80%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.82,83.35,97.14%,93.39%,95.27%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,80.93,77.29,80.86%,66.75%,73.81%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.81,83.80,96.25%,92.51%,94.38%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,81.18,76.63,81.32%,65.68%,73.50%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,80.97,77.07,81.22%,66.06%,73.64%
9,Pena High School,Charter,962,"$585,858.00",$609.00,84.04,83.84,95.95%,94.59%,95.27%


* Create a dataframe to hold the above results

In [24]:
df_schoolsummarytable = pd.DataFrame({"School Name":schoolsummarytable['School Name'],
                                      "Type":schoolsummarytable['Type'],
                                      "Budget":schoolsummarytable['Budget'],
                                      "Total Students":schoolsummarytable['Total Students'],
                                      "Per Student Budget":schoolsummarytable['Per Student Budget'],
                                      "Average Math Score":schoolsummarytable['Average Math Score'],
                                      "Average Reading Score":schoolsummarytable['Average Reading Score'],
                                      "% Passing Math":schoolsummarytable['% Passing Math'],
                                      "% Passing Reading":schoolsummarytable['% Passing Reading'],
                                      "% Overal Passing Rate":schoolsummarytable['% Overall Passing Rate']                                   
                                      })
df_schoolsummarytable

,School Name,Type,Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overal Passing Rate
0,Bailey High School,District,"$3,124,928.00","4,976",$628.00,77.05,81.03,66.68%,81.93%,74.31%
1,Cabrera High School,Charter,"$1,081,356.00","1,858",$582.00,83.06,83.98,94.13%,97.04%,95.59%
2,Figueroa High School,District,"$1,884,411.00","2,949",$639.00,76.71,81.16,65.99%,80.74%,73.36%
3,Ford High School,District,"$1,763,916.00","2,739",$644.00,77.10,80.75,68.31%,79.30%,73.80%
4,Griffin High School,Charter,"$917,500.00","1,468",$625.00,83.35,83.82,93.39%,97.14%,95.27%
5,Hernandez High School,District,"$3,022,020.00","4,635",$652.00,77.29,80.93,66.75%,80.86%,73.81%
6,Holden High School,Charter,"$248,087.00",427,$581.00,83.80,83.81,92.51%,96.25%,94.38%
7,Huang High School,District,"$1,910,635.00","2,917",$655.00,76.63,81.18,65.68%,81.32%,73.50%
8,Johnson High School,District,"$3,094,650.00","4,761",$650.00,77.07,80.97,66.06%,81.22%,73.64%
9,Pena High School,Charter,"$585,858.00",962,$609.00,83.84,84.04,94.59%,95.95%,95.27%


## 3) Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [25]:
TopPerformSchool = schoolsummarytable
TopPerformSchool = TopPerformSchool\
                        .sort_values(["% Overall Passing Rate"] ,ascending=False)\
                        .set_index(['School Name'])\
                        .head(5)

### @@Top Performing Schools (By Passing Rate) - Results:

In [26]:
TopPerformSchool

,Type,Total Students,Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.98,83.06,97.04%,94.13%,95.59%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.85,83.42,97.31%,93.27%,95.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.82,83.35,97.14%,93.39%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,84.04,83.84,95.95%,94.59%,95.27%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.99,83.27,96.54%,93.87%,95.20%


## 4) Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [27]:
BottomPerformSchool = schoolsummarytable
BottomPerformSchool = BottomPerformSchool\
                            .sort_values(["% Overall Passing Rate"],ascending=True)\
                            .set_index(['School Name'])\
                            .head(5)

### @@Bottom Performing Schools (By Passing Rate) - Results:

In [28]:
BottomPerformSchool

,Type,Total Students,Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,80.74,76.84,80.22%,66.37%,73.29%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.16,76.71,80.74%,65.99%,73.36%
Huang High School,District,"2,917","$1,910,635.00",$655.00,81.18,76.63,81.32%,65.68%,73.50%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,80.97,77.07,81.22%,66.06%,73.64%
Ford High School,District,"2,739","$1,763,916.00",$644.00,80.75,77.10,79.30%,68.31%,73.80%


## 5) Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  * Group each series by school

In [29]:
#First checking what values we have for "grade" in the table
grades=school_data_complete['grade'].unique()
grades

array(['9th', '12th', '11th', '10th'], dtype=object)

In [30]:
#Create the series for each grade using conditional statements, then group by school.
MathScoresByGrade9th = school_data_complete.loc[school_data_complete['grade'] == '9th', ['school_name','math_score']]
MathScoresByGrade9thMean = MathScoresByGrade9th.groupby('school_name')['math_score'].mean()

MathScoresByGrade10th = school_data_complete.loc[school_data_complete['grade'] == '10th', ['school_name','math_score']]
MathScoresByGrade10thMean = MathScoresByGrade10th.groupby('school_name')['math_score'].mean()

MathScoresByGrade11th = school_data_complete.loc[school_data_complete['grade'] == '11th', ['school_name','math_score']]
MathScoresByGrade11thMean = MathScoresByGrade11th.groupby('school_name')['math_score'].mean()

MathScoresByGrade12th = school_data_complete.loc[school_data_complete['grade'] == '12th', ['school_name','math_score']]
MathScoresByGrade12thMean = MathScoresByGrade12th.groupby('school_name')['math_score'].mean()

  * Combine the series into a dataframe

In [31]:
mathscores = pd.DataFrame({"9th":MathScoresByGrade9thMean,
                           "10th":MathScoresByGrade10thMean,
                           "11th":MathScoresByGrade11thMean,
                          "12th":MathScoresByGrade12thMean
                          })

* Optional: give the displayed data cleaner formatting

In [32]:
mathscores['9th'] = mathscores['9th'].map("{:,.2f}".format)
mathscores['10th'] = mathscores['10th'].map("{:,.2f}".format)
mathscores['11th'] = mathscores['11th'].map("{:,.2f}".format)
mathscores['12th'] = mathscores['12th'].map("{:,.2f}".format)
mathscores = mathscores.rename_axis('School Name')


### @@ Math Scores by Grade - Results:

In [33]:
mathscores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## 6) Reading Score by Grade 

* Perform the same operations as above for reading scores

In [34]:
ReadingScoresByGrade9th = school_data_complete.loc[school_data_complete['grade'] == '9th', ['school_name','reading_score']]
ReadingScoresByGrade9thMean = ReadingScoresByGrade9th.groupby('school_name')['reading_score'].mean()

ReadingScoresByGrade10th = school_data_complete.loc[school_data_complete['grade'] == '10th', ['school_name','reading_score']]
ReadingScoresByGrade10thMean = ReadingScoresByGrade10th.groupby('school_name')['reading_score'].mean()

ReadingScoresByGrade11th = school_data_complete.loc[school_data_complete['grade'] == '11th', ['school_name','reading_score']]
ReadingScoresByGrade11thMean = ReadingScoresByGrade11th.groupby('school_name')['reading_score'].mean()

ReadingScoresByGrade12th = school_data_complete.loc[school_data_complete['grade'] == '12th', ['school_name','reading_score']]
ReadingScoresByGrade12thMean = ReadingScoresByGrade12th.groupby('school_name')['reading_score'].mean()

readingscores = pd.DataFrame({"9th":ReadingScoresByGrade9thMean,
                           "10th":ReadingScoresByGrade10thMean,
                           "11th":ReadingScoresByGrade11thMean,
                          "12th":ReadingScoresByGrade12thMean
                          })

In [35]:
readingscores['9th'] = readingscores['9th'].map("{:,.2f}".format)
readingscores['10th'] = readingscores['10th'].map("{:,.2f}".format)
readingscores['11th'] = readingscores['11th'].map("{:,.2f}".format)
readingscores['12th'] = readingscores['12th'].map("{:,.2f}".format)
readingscores = readingscores.rename_axis('School Name')

### @@ Reading Scores by Grade - Results:

In [36]:
readingscores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## 7) Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [37]:
# Based on the sample, felt like the bins/labels proposed made sense to keep for this analysis.
spending_bins = [0, 585, 620, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [38]:
ScoreBySchoolSpending = df_temp
ScoreBySchoolSpending.columns

Index(['school_name', 'Type', 'Total Students', 'Budget', 'Per Student Budget',
       'Average Reading Score', 'Average Math Score', '% Passing Reading',
       '% Passing Math', '% Overall Passing Rate'],
      dtype='object')

In [39]:
ScoreBySchoolSpending["Spending ranges"] = pd.cut(ScoreBySchoolSpending["Per Student Budget"],\
                                                  spending_bins,labels=group_names)
ScoreBySchoolSpending = ScoreBySchoolSpending.drop(['Type', 'Total Students','Budget','Per Student Budget'],axis=1)

In [40]:
ScoreBySchoolSpending = ScoreBySchoolSpending.groupby('Spending ranges').mean()

### @@Scores by School Spending - Results:

In [41]:
formarter(ScoreBySchoolSpending,0)

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Spending ranges,,,,,
<$585,83.93,83.46,96.61%,93.46%,95.04%
$585-615,83.89,83.60,95.90%,94.23%,95.07%
$615-645,81.89,79.08,86.11%,75.67%,80.89%
$645-675,81.03,77.00,81.13%,66.16%,73.65%


## 8) Scores by School Size

* Perform the same operations as above, based on school size.

In [42]:
# Felt like the bins/labels proposed made sense to keep for this analysis, just adjusted the Medium size.
size_bins = [0, 1000, 3000, 5000]
group_names2 = ["Small (<1000)", "Medium (1000-3000)", "Large (3000-5000)"]

In [43]:
ScoreBySchoolSize = df_temp

In [44]:
ScoreBySchoolSize["Size ranges"] = pd.cut(ScoreBySchoolSize["Total Students"],size_bins,labels=group_names2)
ScoreBySchoolSize = ScoreBySchoolSize.drop(['Type', 'Total Students','Budget','Per Student Budget'],axis=1)
ScoreBySchoolSize = ScoreBySchoolSize.groupby('Size ranges').mean()

## @@Scores by School Size - Results:

In [45]:
formarter(ScoreBySchoolSize,0)

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
Size ranges,,,,,
Small (<1000),83.93,83.82,96.10%,93.55%,94.82%
Medium (1000-3000),82.93,81.18,91.32%,84.65%,87.98%
Large (3000-5000),80.92,77.06,81.06%,66.46%,73.76%


## 9) Scores by School Type

* Perform the same operations as above, based on school type.

In [46]:
ScoreBySchoolType = df_temp

In [47]:
ScoreBySchoolType = ScoreBySchoolType.drop(['Total Students','Budget','Per Student Budget'],axis=1)

In [48]:
ScoreBySchoolType = ScoreBySchoolType.groupby('Type').mean()
ScoreBySchoolType = ScoreBySchoolType.rename_axis('School Type')

## @@Scores by School Type - Results:

In [49]:
formarter(ScoreBySchoolType,0)

,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing Rate
School Type,,,,,
Charter,83.90,83.47,96.59%,93.62%,95.10%
District,80.97,76.96,80.80%,66.55%,73.67%


## @@ Small piece of additional analysis

In [50]:
#Wanted to check if there were any relevant differences based on gender.

additionaltable = pd.pivot_table(school_data_complete,values =['student_name','math_score','reading_score','size'],\
                       index=['type', 'gender'],\
                       aggfunc={'math_score':np.mean,
                                'reading_score': np.mean,
                                'gender': len,
                                'size': np.mean})

#Noticed no relevant differences at all, instead confirmed that gender does not play a difference in this dataset.
display (additionaltable.head(), additionaltable.describe())

gender  math_score  reading_score         size
type     gender                                                
Charter  F         6184   83.369502      83.813713  1715.824547
         M         6010   83.443927      83.994509  1718.924626
District F        13551   77.062578      81.166925  4073.863774
         M        13425   76.910764      80.756127  4052.559106

,gender,math_score,reading_score,size
count,4.000000,4.000000,4.000000,4.000000
mean,9792.500000,80.196693,82.432818,2890.293013
std,4268.097039,3.707256,1.708755,1354.398057
min,6010.000000,76.910764,80.756127,1715.824547
25%,6140.500000,77.024625,81.064225,1718.149606
50%,9804.500000,80.216040,82.490319,2885.741866
75%,13456.500000,83.388108,83.858912,4057.885273
max,13551.000000,83.443927,83.994509,4073.863774
